# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [82]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [83]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [84]:
# Your code here
scaler = StandardScaler()
# Scale X_train and X_val using StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)
# Ensure X_train and X_val are scaled DataFrames
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)
# (hint: you can set the columns using X.columns)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotArea       821 non-null    float64
 1   OverallQual   821 non-null    float64
 2   OverallCond   821 non-null    float64
 3   TotalBsmtSF   821 non-null    float64
 4   1stFlrSF      821 non-null    float64
 5   2ndFlrSF      821 non-null    float64
 6   GrLivArea     821 non-null    float64
 7   TotRmsAbvGrd  821 non-null    float64
 8   GarageArea    821 non-null    float64
 9   Fireplaces    821 non-null    float64
dtypes: float64(10)
memory usage: 64.3 KB


In [51]:
# Your code here
regression = LinearRegression()
# Create a LinearRegression model and fit it on scaled training data
regression.fit(X_train, y_train)
# Calculate a baseline r-squared score on training data
baseline_train = regression.score(X_train, y_train)
baseline_val = regression.score(X_val, y_val)

baseline_train, baseline_val

(0.7868344817421309, 0.6508330438442944)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [52]:
# Your code here

# Set up data structure
interactions = []
# Find combinations of columns and loop over them
column_combos = list(combinations(X_train.columns, 2))
    # Make copies of X_train and X_val
for (col1, col2) in column_combos:
    feature_train = X_train.copy()
    feature_val = X_val.copy()
    
    # Add interaction term to data

    feature_train['interaction'] = feature_train[col1] * feature_train[col2]
    feature_val['interaction'] = feature_val[col1] * feature_val[col2]
    # Find r-squared score (fit on training data, evaluate on validation data)
    score = LinearRegression().fit(feature_train, y_train).score(feature_val, y_val)
    
    # Append to data structure
    interactions.append(((col1,col2), score))
    
# Sort and subset the data structure to find the top 7
top_seven_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
top_seven_interactions

[(('LotArea', '1stFlrSF'), 0.7626719257427556),
 (('LotArea', 'TotalBsmtSF'), 0.7457308916478046),
 (('LotArea', 'GrLivArea'), 0.7004211108080429),
 (('LotArea', 'Fireplaces'), 0.6726762646471813),
 (('2ndFlrSF', 'TotRmsAbvGrd'), 0.661349767775135),
 (('OverallCond', 'TotalBsmtSF'), 0.6559227960585219),
 (('OverallQual', '2ndFlrSF'), 0.655894468087473)]

### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [53]:
# Your code here
for record in top_seven_interactions:
# Loop over top 7 interactions
    col1,col2 = record[0]
    # Extract column names from data structure
    new_col_name = col1+ " " + col2
    # Construct new column name
    X_train[new_col_name] = X_train[col1]*X_train[col2]
    X_val[new_col_name] = X_train[col1] * X_train[col2]
    # Add new column to X_train and X_val
X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea 1stFlrSF,LotArea TotalBsmtSF,LotArea GrLivArea,LotArea Fireplaces,2ndFlrSF TotRmsAbvGrd,OverallCond TotalBsmtSF,OverallQual 2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,0.477622,0.271823,0.720306,-0.313023,1.789339,0.260039,0.080686
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.097522,-0.158985,0.148966,0.307643,0.295392,-0.261809,0.080686
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.107471,-0.061263,0.095069,0.070221,1.460730,0.261252,1.064983
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,0.003613,0.002418,0.000789,0.002704,-0.286483,-1.160508,-0.078252


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [54]:
# Your code here

# Set up data structure
polynomials = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2,3,4):
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(features_val[[col]]))
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val], axis=1)
    
        # Find r-squared score on validation
        score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
        # Append to data structure
        polynomials.append((col, degree, score))
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7] 

top_7_polynomials

[('GrLivArea', 4, 0.8157200978366146),
 ('TotalBsmtSF', 3, 0.7356173652161555),
 ('GrLivArea', 3, 0.7305812505698422),
 ('LotArea', 3, 0.6758625050956081),
 ('2ndFlrSF', 2, 0.6512784821920666),
 ('2ndFlrSF', 4, 0.6431324906802858),
 ('2ndFlrSF', 3, 0.6415177488168156)]

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [61]:
# Your code here
top_polynomials = pd.DataFrame(top_7_polynomials, columns=["Column", "Degree", "R^2"])
top_polynomials
# Filter out duplicates
top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials
# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                               columns=poly.get_feature_names([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                               columns=poly.get_feature_names([col]))
    # Concat new polynomials to X_train and X_val
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

In [62]:
top_polynomials

,Column,Degree,R^2
0,GrLivArea,4,0.815720
1,TotalBsmtSF,3,0.735617
3,LotArea,3,0.675863
4,2ndFlrSF,2,0.651278


Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [64]:
# Your code here
X_train.head()

,OverallQual,OverallCond,1stFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea 1stFlrSF,LotArea TotalBsmtSF,LotArea GrLivArea,LotArea Fireplaces,...,GrLivArea^3,GrLivArea^4,TotalBsmtSF,TotalBsmtSF^2,TotalBsmtSF^3,LotArea,LotArea^2,LotArea^3,2ndFlrSF,2ndFlrSF^2
0,-0.099842,-0.509252,-0.804789,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,...,0.124337,0.062058,-0.639316,0.408725,-0.261304,-0.114710,0.013158,-0.001509,1.261552,1.591512
1,0.632038,-0.509252,0.641608,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,...,-0.015115,0.003737,0.838208,0.702592,0.588918,-0.176719,0.031230,-0.005519,-0.808132,0.653077
2,-0.831723,1.304613,-0.329000,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,...,-0.843282,0.796704,-0.012560,0.000158,-0.000002,-0.246336,0.060682,-0.014948,-0.808132,0.653077
3,-0.831723,1.304613,-0.609036,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,...,-1.505101,1.724862,-0.339045,0.114951,-0.038974,-0.378617,0.143351,-0.054275,-0.808132,0.653077
4,-1.563603,1.304613,-1.315922,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,...,-0.111777,0.053844,-2.531499,6.408489,-16.223084,-0.010898,0.000119,-0.000001,0.550523,0.303075


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [65]:
# Your code here
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train R^2: ", lr.score(X_train,y_train))
print("Validation R^2: ", lr.score(X_val, y_val))

Train R^2:  0.830744952038789
Validation R^2:  0.7719343410538894


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [69]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)
    
    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)
    print("Train R^2: ", lr.score(X_rfe_train, y_train))
    print("Test R^2 :", lr.score(X_rfe_val, y_val))
    print(f"Using {n} out of {X_train.shape[1]} features")
    print()

Train R^2:  0.726643976914122
Test R^2 : 0.4949478920434889
Using 5 out of 49 features

Train R^2:  0.7865090619899151
Test R^2 : 0.5406760004875079
Using 10 out of 49 features

Train R^2:  0.806850837905162
Test R^2 : 0.36541055497070896
Using 15 out of 49 features

Train R^2:  0.8098777022752223
Test R^2 : 0.3983500948330011
Using 20 out of 49 features

Train R^2:  0.8213924207705646
Test R^2 : 0.6098865954333101
Using 25 out of 49 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [76]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    print("Train R^2: ", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of ", alpha)
    print()

Train R^2:  0.8307447254672586
Validation R^2:  0.772209383449114
Using 26 out of 49 features
and an alpha of  1

Train R^2:  0.8307269876062462
Validation R^2:  0.7742368597185783
Using 24 out of 49 features
and an alpha of  10

Train R^2:  0.8305779925054706
Validation R^2:  0.7832108294197899
Using 24 out of 49 features
and an alpha of  100

Train R^2:  0.8229941868521051
Validation R^2:  0.8253752088545592
Using 14 out of 49 features
and an alpha of  1000

Train R^2:  0.7447764334885001
Validation R^2:  0.4461146361522126
Using -3 out of 49 features
and an alpha of  10000



Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here
"""
I would be using the one with 12 features. It has the highest R^2 value and explains the most of the test data
"""

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [85]:
# Your code here
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

for record in top_seven_interactions:
    col1, col2 = record[0]
    new_col_name = col1 + " " + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]
    
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                               columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [86]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 25)

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.